<a href="https://colab.research.google.com/github/gibranfp/CursoDatosMasivosI/blob/main/notebooks/4c_elementos_distintos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo HyperLogLog
En esta libreta programaremos el algoritmo HyperLogLog  para estimar el número de elementos distintos en un flujo de datos.

La idea detrás de este algoritmo es que entre más elementos diferentes haya en el flujo de datos, más valores _hash_ diferentes veremos. A medida que vemos más valores _hash_ diferentes, es más probable que uno de estos valores sea inusual, en este caso que termine en muchos ceros.

En particular, sea $R$ el número de 0s al final de la cadena correspondiente al valor _hash_ de algún elemento, se mantiene el $R$ más grande de los elementos del flujo $\hat{R}$ y se toma $2^\hat{R}$ como un estimador del número de elementos distintos.

In [ ]:
import numpy as np
np.random.seed(2021) # para reproducibilidad
import murmurhash

class ConteoProbabilista:  
  def __init__(self, n_cubetas, primo):
    self.primo = primo  
    self.n_cubetas = n_cubetas
    self.tam_max = 0
    self.a = np.random.randint(1, self.primo - 1)
    self.b = np.random.randint(0, self.primo - 1)
    
  def __call__(self, x):
    hv = ((self.a * x + self.b) % self.primo) % self.n_cubetas
    tam = bin(hv)[2:][::-1].find('1')
    if self.tam_max < tam:
      self.tam_max = tam

  def cardinalidad(self):
    return 2**self.tam_max

Definimos una clase que realiza varias estimaciones, las divide en grupos pequeños, obtiene el promedio de las estimaciones de cada grupo y toma la mediana de estos promedios como estimación final.

In [ ]:
class EstimadorElementosDistintos:
  def __init__(self, n_cubetas, n_grupos, n_funciones, primo):
    self.n_grupos = n_grupos
    self.n_funciones = n_funciones
    self.estimadores = []
    for i in range(self.n_grupos):
      func = []
      for j in range(self.n_funciones):
        func.append(ConteoProbabilista(n_cubetas, primo))
      self.estimadores.append(func)
    self.conteos = np.zeros((self.n_grupos, self.n_funciones))

  def __call__(self, x):
    for i in range(self.n_grupos):
      for j in range(self.n_funciones):  
        self.estimadores[i][j](x)
        self.conteos[i, j] = self.estimadores[i][j].cardinalidad()
      
  def cardinalidad(self):
    return np.median(self.conteos.mean(axis=0))

Generamos números aleatorios.

In [ ]:
import numpy as np

X = np.random.randint(0,10000, size=100000)
print("Hay {0} elementos distintos".format(np.unique(X).size))

Hay 10000 elementos distintos


Instanciamos nuestra clase y estimamos elementos distintos.

In [ ]:
est = EstimadorElementosDistintos(100000, 2, 30, 104729)
for i,x in enumerate(X):
  est(x)
print(est.cardinalidad())

12288.0
